<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> COP-4</h1>
</center>

## <span style="color:blue">Nombre</span>: Francis Vargas Ferrer

## <span style="color:blue">Rol</span>: 201573026-1

# Librerías

In [1]:
import numpy as np
from scipy.linalg import solve_triangular

# Desarrollo

Método de $SOR(\omega)$ obtenido del material de github[1], se le modificaron los parámetros que recibe para que sean los del problema.

In [2]:
def SOR(A, b, w=1.1, n_iter=3, x_0=None):
    n = A.shape[0] # Matrix size
    X = np.zeros((n_iter + 1, n)) # Matrix with solution at each iteration
    # Initial guess
    if x_0 is not None:
        X[0] = x_0
    L = np.tril(A, k=-1) # Get lower triangle 
    U = np.triu(A, k=1) # Get Upper triangle 
    D = A - U - L
    # SOR
    for k in range(n_iter):
        X[k+1] = solve_triangular(w * L + D, w * b + np.dot((1 - w) * D - w * U, X[k]), lower=True)
    return X

Definiremos nuestro $F(\beta)$

In [3]:
def get_F(x, y):
    f1 = lambda beta: np.sum((y-beta[0]-np.sin(beta[1]*x)-np.cos(beta[2]*x))**3)
    f2 = lambda beta: np.sum(x*np.cos(beta[1]*x)*(y-beta[0]-np.sin(beta[1]*x)-np.cos(beta[2]*x))**3)
    f3 = lambda beta: np.sum(x*np.sin(beta[2]*x)*(y-beta[0]-np.sin(beta[1]*x)-np.cos(beta[2]*x))**3)
    return lambda beta: np.array([f1(beta),f2(beta),f3(beta)])

Creamos la función que nos entrega el resultado de $J(\beta)v$

In [4]:
def J(F, beta, epsilon=1e-10):
    ev = np.array([
        [epsilon, 0, 0],
        [0, epsilon, 0],
        [0, 0, epsilon]
    ])
    J = np.array([(F(beta+ev[0])-F(beta))/epsilon, 
                  (F(beta+ev[1])-F(beta))/epsilon,
                  (F(beta+ev[2])-F(beta))/epsilon]).T
    #Usamos la transpuesta ya que sin esto cada fila es la columna de J
    return J

Creamos el método de Levenberg-Marquardt, el cual es una modificación del método de Newton usado en el curso[2]. El cambio ocurre en la línea 5 y 6, donde para resolver el sistema se utiliza el método $SOR(\omega)$, el cual retorna todas las soluciones, como nos interesa la última, en la línea 6 solo se utiliza el último elemento de w.

In [5]:
def LM(F, b_0, n, tol=1e-10, lambd=1):
    b = np.zeros((n + 1, b_0.shape[0]))
    b[0] = b_0
    for k in range(n):
        w = SOR(J(F, b[k]).T@J(F, b[k]) + lambd*np.eye(b_0.shape[0]), -J(F, b[k]).T@F(b[k]))
        b[k+1] = b[k] + w[-1]
        if np.linalg.norm(F(b[k+1])) < tol:
            b = b[:k+2]
            break
    return b

Como tenemos todo definido, aplicaremos el método a los valores entregados.

In [6]:
#Cargamos el dataset
data = np.load("1.npy")
x = data[:,0]
y = data[:,1]

#Definimos B_0
b_0 = np.array([1,1,1])

#Obtenemos F
F = get_F(x, y)
#Aplicamos el método
B = LM(F, b_0, 3)

#### 1 ¿Cuál es el valor de la segunda componente de $F(\beta_0)$?

In [7]:
F(b_0)[1]

-1332.0028850739136

#### 2  ¿Cuál es la norma de Frobenius de la aproximación del Jacobiano en $\beta_0$, es decir,  $∥J(\beta_0)∥_F$?

In [8]:
np.linalg.norm(J(F, b_0), ord='fro')

16783.581619549197

#### ¿Cuál es el valor del parámetro $\beta_1$ luego de ajustar el modelo?

In [9]:
B[-1][0]

1.0739414958375249

# Referencias:
[1] https://github.com/sct-utfsm/INF-285/blob/master/material/04_sistemas_ecuaciones/metodos_iterativos.ipynb


[2]https://github.com/sct-utfsm/INF-285/blob/master/material/04_sistemas_ecuaciones/no_lineal.ipynb